# # OSM etymology data for verification

## Get OSM data
Query all streets with existing etymology designation in Switzerland

In [1]:
#Load all needed libraries
import requests
import pandas as pd
import osm2geojson
import geopandas as gpd
from time import process_time_ns
import numpy as np
from SPARQLWrapper import SPARQLWrapper, JSON

Testing your query for needed OSM informations: https://overpass-turbo.osm.ch/

Data model for streets (topic : highway): https://wiki.openstreetmap.org/wiki/Key%3ahighway

<font color='red'>WARNING</font> takes a couple of minutes to retrieve data for entire Switzerland

<font color='blue'>IMPORTANT</font> filter only name:etymology:wikidata (https://wiki.openstreetmap.org/wiki/Key:name:etymology:wikidata)

In [2]:
#overpass_url = "http://overpass-api.de/api/interpreter"
overpass_url = "https://lz4.overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["ISO3166-1"="CH"];
(
nwr["name:etymology:wikidata"]["highway"](area);
);
(._;>;);
out body;
"""
response = requests.get(overpass_url, data = overpass_query)
print(response)

<Response [200]>


In [3]:
data = response.json()
#data

In [4]:
dfp = pd.DataFrame(data["elements"])
dfp

,type,id,lat,lon,tags,nodes,members
0,node,172271,46.524137,6.594365,NaN,NaN,NaN
1,node,172272,46.524011,6.595446,NaN,NaN,NaN
2,node,172273,46.523461,6.597860,NaN,NaN,NaN
3,node,172274,46.523198,6.599370,NaN,NaN,NaN
4,node,280590,46.516906,6.615215,NaN,NaN,NaN
...,...,...,...,...,...,...,...
31237,relation,3400183,NaN,NaN,"{'highway': 'pedestrian', 'layer': '5', 'name'...",NaN,"[{'type': 'way', 'ref': 68687544, 'role': 'inn..."
31238,relation,4800640,NaN,NaN,"{'area': 'yes', 'highway': 'pedestrian', 'lit'...",NaN,"[{'type': 'way', 'ref': 86090007, 'role': 'out..."
31239,relation,8437762,NaN,NaN,"{'alt_name': 'Le Corbusier-Platz', 'highway': ...",NaN,"[{'type': 'way', 'ref': 606423418, 'role': 'ou..."
31240,relation,8688199,NaN,NaN,"{'bicycle': 'yes', 'highway': 'pedestrian', 'l...",NaN,"[{'type': 'way', 'ref': 343989936, 'role': 'ou..."


In [5]:
geojson = osm2geojson.json2geojson(data)
#geojson

In [6]:
geom = gpd.GeoDataFrame.from_features(geojson, crs="EPSG:4326")
geom

,geometry,type,id,tags,nodes
0,POINT (8.25699 46.89458),node,40128715,"{'highway': 'motorway_junction', 'name': 'Sarn...",NaN
1,POINT (8.43598 46.97224),node,249282757,"{'highway': 'motorway_junction', 'name': 'Buoc...",NaN
2,POINT (8.34915 46.96803),node,292003425,"{'highway': 'motorway_junction', 'name': 'Stan...",NaN
3,POINT (7.43698 46.94589),node,443705798,"{'addr:city': 'Bern', 'addr:country': 'CH', 'a...",NaN
4,POINT (7.43679 46.94590),node,443705802,"{'addr:city': 'Bern', 'addr:country': 'CH', 'a...",NaN
...,...,...,...,...,...
4317,"MULTILINESTRING ((7.42169 46.94472, 7.42177 46...",relation,7757,"{'highway': 'secondary', 'name': 'Effingerstra...",NaN
4318,"MULTIPOLYGON (((6.63465 46.52260, 6.63461 46.5...",relation,1715790,"{'area': 'yes', 'highway': 'pedestrian', 'name...",NaN
4319,"MULTIPOLYGON (((6.63061 46.52095, 6.63049 46.5...",relation,4800640,"{'area': 'yes', 'highway': 'pedestrian', 'lit'...",NaN
4320,"MULTIPOLYGON (((7.37492 46.94708, 7.37492 46.9...",relation,8437762,"{'alt_name': 'Le Corbusier-Platz', 'highway': ...",NaN


In [7]:
info = pd.json_normalize(geom.to_dict('list'), ['tags'])
pd.options.display.max_rows = 50
pd.options.display.max_columns = 300
info.head(5)

,highway,name,name:etymology:wikidata,ref,addr:city,addr:country,addr:housenumber,addr:postcode,addr:street,entrance,public_transport,trolleybus,alt_name,brand,bus,ele,network,operator,shelter,source,uic_name,uic_ref,website,wheelchair,bench,bin,lit,tactile_paving,wlan,covered,network:wikidata,cycleway:both,lane_markings,maxspeed,sidewalk,surface,trolley_wire,motor_vehicle,oneway,cycleway,oneway:bicycle,lanes,lcn,note,owner,smoothness,wikidata,foot,name:etymology:wikipedia,bicycle:backward,bicycle:left,bus:lanes:backward,cycleway:right,lanes:backward,lanes:forward,old_name,vehicle:lanes:backward,turn:lanes:backward,turn:lanes:forward,bicycle:forward,bridge,cycleway:left,layer,maxweight:signed,motorcar:conditional,motorcycle:conditional,access,vehicle,sidewalk:both:surface,incline,noexit,shoulder,psv,sidewalk:left,sidewalk:right,check_date:cycleway,cycleway:left:lane,cycleway:left:oneway,image,name:source,name:start_date,wikimedia_commons,rcrc_ref,access:lanes:backward,bicycle:lanes:backward,cycleway:lanes:backward,official_name,junction,cycleway:right:lane,lanes:unmarked,mapillary,service,bicycle,dog,maxspeed:type,source:maxspeed,fixme,source:width,width,lanes:psv:backward,tunnel,cycleway:both:lane,access:conditional,wikipedia,lanes:psv:forward,source:name,maxheight,maxweight,turn:lanes,nat_ref,reg_ref,bicycle:lanes:forward,cycleway:lanes:forward,motor_vehicle:lanes:forward,motorcar,motorcycle,destination:backward,destination:lanes,day_off,day_on,hour_off,hour_on,tracktype,oneway:bus,oneway:moped,oneway:psv,access:lanes:forward,parking:lane:both,segregated,taxi,horse,handrail:left,ramp,step_count,sidewalk:left:surface,parking:lane:right,parking:lane:right:parallel,loc_ref,mofa,mofa:backward,motor_vehicle:backward,psv:backward,psv:lanes:forward,area,busway:left,lanes:psv,psv:lanes,parking:lane:left,parking:lane:left:parallel,cyclestreet,old_name:-1934,sac_scale,vehicle:lanes,sidewalk:right:surface,railway,hgv,handrail,busway:right,cycleway:right:oneway,oneway:foot,embankment,crossing,cycleway:surface,footway:surface,motor_vehicle:conditional,access:backward,short_name,crossing:island,footway,maxlength,sidewalk:both,destination:forward,destination:symbol:forward,start_date,disabled,destination,check_date:surface,agricultural,placement,oneway:hgv,bridge:name,parking:condition:right,parking:condition:right:maxstay,parking:condition:right:residents,level,check_date,proposed:trolley_wire,car,traffic_calming,vehicle:lanes:forward,oneway:mofa,postal_code,old_name:-2006,parking:lane,name:etymology,start_date:name,ramp:wheelchair,moped,handrail:right,trolley_wire:backward,psv:lanes:backward,parking:condition:both,parking:condition:both:maxstay,parking:condition:both:residents,avz,cs_dir:backward,cs_dir:forward,destination:symbol,maxspeed:backward,maxspeed:forward,lanes:directions,handrail:center,ford,mtb:scale,mtb:scale:imba,mtb:scale:uphill,trail_visibility,place,turn,loc_name,parking:condition:right:2,parking:condition:right:2:maxstay,parking:condition:right:2:time_interval,parking:condition:right:3,parking:condition:right:3:time_interval,parking:condition:right:default,parking:condition:right:time_interval,parking:lane:right:capacity,description,departures_board,overtaking,destination:lanes:backward,kerb:left,noname,cycleway:lane,pedestrian_lane,check_date:ramp,parking:condition:left,parking:lane:left:capacity,traffic_sign,trailer,cycleway:left:segregated,was:foot,bus:backward,type
0,motorway_junction,Sarnen Nord,Q63964,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [8]:
most_common = info.groupby("highway").count()
most_common.name.nlargest(10)

highway
residential      1562
footway           533
secondary         471
tertiary          381
primary           304
service           273
living_street     209
unclassified      198
pedestrian        134
steps              96
Name: name, dtype: int64

In [9]:
verif = info[['name','name:etymology:wikidata']]
verif

,name,name:etymology:wikidata
0,Sarnen Nord,Q63964
1,Buochs,Q64567
2,Stans Nord,Q63931
3,NaN,Q1296211
4,NaN,Q1296211
...,...,...
4317,Effingerstrasse,Q1296211
4318,Place de la Cathédrale,Q669544
4319,Place de l'Europe,Q46
4320,Le-Corbusier-Platz,Q4724


In [10]:
#Drop when missing data
missing_data = pd.DataFrame(
    verif.isnull().sum(),
    columns=['Missing Values'])

print(missing_data)

verif = verif.dropna()

                         Missing Values
name                                 19
name:etymology:wikidata               0


In [11]:
#Drop duplicates
uniqueValues = verif.drop_duplicates()
uniqueValues

,name,name:etymology:wikidata
0,Sarnen Nord,Q63964
1,Buochs,Q64567
2,Stans Nord,Q63931
12,Tavelweg,Q124667
15,Willadingweg,Q1693700
...,...,...
3991,Hintere Erlachstrasse,Q123886
3992,Hardungweg,Q1495926
3993,Spühlweg,Q78071721;Q106809423
3994,Spühlstrasse,Q78071721;Q106809423


In [12]:
duplicateRows = uniqueValues[uniqueValues.duplicated(['name'])]
duplicateRows

,name,name:etymology:wikidata
509,Avenue de la Gare,Q669678
894,Weberstrasse,Q55903732
964,Schillerstrasse,Q107030305;Q22670
979,Place de la Gare,Q669678
1111,Simonstrasse,Q23063169
1351,Fröbelstrasse,Q76679
1758,Rue de la Tour,Q3533300
1775,Schlatterstrasse,Q43136553
2173,Hiltystrasse,Q55196805;Q120960


Drop these 9 non unique names

In [13]:
uniqueValues = uniqueValues.drop_duplicates(subset=['name'])

Only keep first QID per name

In [14]:
uniqueValues['name:etymology:wikidata'] = uniqueValues['name:etymology:wikidata'].str.replace("(;).*","", regex=True)

C:\Users\micha\AppData\Local\Temp\ipykernel_4972\3111103421.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqueValues['name:etymology:wikidata'] = uniqueValues['name:etymology:wikidata'].str.replace("(;).*","", regex=True)


Rename columns

In [15]:
uniqueValues.columns = ['name', 'qid']

# Add instanceLabel, sex and dateOfBirth from wikidata 

<font color='blue'>IMPORTANT</font> We need that instance (class) of the "thing" the street is named after to be able to classify. Data was not found via OSM.

Testing Queries under https://query.wikidata.org/#

In [16]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

In [17]:
def queryInstance(sparql, subject):
    #print(wdKey)
    query = """
    SELECT  ?item ?instanceLabel ?dateOfBirth ?sexLabel ?pic
    WHERE
    {
      ?item wdt:P31 ?instance.
      FILTER(?item = wd:%s)
      OPTIONAL {?item wdt:P569 ?dateOfBirth  . }
      OPTIONAL {?item wdt:P21 ?sex  . }
      OPTIONAL {?item wdt:P18 ?pic  . }
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE]". }
    }
    LIMIT 1
    """ % (subject)
    #print(query)
    
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql

In [18]:
def get_retry_after():
    response = requests.get(wdUrl)
    retry_after = response.getheader("Retry-After")

    if retry_after is None:
        return None

    print(retry_after)

<font color='red'>WARNING</font> takes about 10 minutes to retrieve data from wikidata

<font color='blue'>IMPORTANT</font> Sollte ein Statuscode 429 auftreten: Anfrage in ca. 30sec wiederholen
HTTP Error 429: Too Many Requests

In [21]:
i = 0
dfr = pd.DataFrame(columns=["qid", "name", "instanceLabel", "dateBirth", "sexLabel", "pic"])

maxresults = 1200

for x in uniqueValues.index:
    i+=1
    #print(i)
    subject = uniqueValues['qid'][x]
    subjectStr = uniqueValues['name'][x]
    #print(f"Subjekt: {subject}")
    sparql = queryInstance(sparql, subject)
    #print(sparql)
    try:
        results = sparql.query()
        #print(results.info())
    except Exception as e:
        #Prüfen auf Statuscode 429 (Too many Requests). Leider kann der Statuscode nicht abgerufen werden..
        print("Sollte ein Statuscode 429 auftreten: Anfrage in ca. 30sec wiederholen")
        print(e)
        get_retry_after()
        break

    result = results.convert()
    #print(result)

    results_df = pd.json_normalize(result['results']['bindings'])
    #print(results_df)

    if not results_df.empty:
        instanceLabel = results_df['instanceLabel.value'][0]
        try:
            dateBirth = results_df['dateOfBirth.value'][0]
        except:
            dateBirth = '9999'
        try:
            sexLabel = results_df['sexLabel.value'][0]
        except:
            sexLabel = 'kein'
        try:
            pic = results_df['pic.value'][0]
        except:
            pic = 'kein'


        #print(f"{subject} | {subjectStr}: {instanceLabel} / {dateBirth} / {sexLabel}  / {pic}")
        dfrtemp = pd.DataFrame([[subject, subjectStr, instanceLabel, dateBirth, sexLabel, pic]], columns=dfr.columns)
        dfr = pd.concat([dfr, dfrtemp])

    else:
        #print(f"{subjectStr} | {subject}: Kein Eintrag in Wikidata gefunden")
        dfrtemp = pd.DataFrame([[subject, subjectStr, np.nan, np.nan, np.nan, np.nan]], columns=dfr.columns)
        dfr = pd.concat([dfr, dfrtemp])

    #print(x)
    if i > maxresults:
        break

In [22]:
dfr.head()

,qid,name,instanceLabel,dateBirth,sexLabel,pic
0,Q63964,Sarnen Nord,Q5119,9999,kein,http://commons.wikimedia.org/wiki/Special:File...
0,Q64567,Buochs,Q70208,9999,kein,http://commons.wikimedia.org/wiki/Special:File...
0,Q63931,Stans Nord,Q70208,9999,kein,http://commons.wikimedia.org/wiki/Special:File...
0,Q124667,Tavelweg,Q5,1866-12-21T00:00:00Z,Q6581097,http://commons.wikimedia.org/wiki/Special:File...
0,Q1693700,Willadingweg,Q5,1641-01-01T00:00:00Z,Q6581097,http://commons.wikimedia.org/wiki/Special:File...


Add sex label

In [23]:
most_common = dfr.groupby("sexLabel").count()
most_common.name.nlargest(10)

sexLabel
Q6581097    668
kein        271
Q6581072    132
Name: name, dtype: int64

In [24]:
sex_dict = {
    'Q6581097': 'male', 
    'Q6581072': 'female', 
    'kein': ''
}

In [25]:
dfr['sex'] = dfr['sexLabel'].map(sex_dict)

Add instance label

In [26]:
most_common = dfr.groupby("instanceLabel").count()
most_common.name.nlargest(150)

instanceLabel
Q5          780
Q16521       49
Q70208       32
Q8502        17
Q8436        17
           ... 
Q3958441      1
Q3977906      1
Q1916821      1
Q1914636      1
Q1569856      1
Name: name, Length: 89, dtype: int64

Manually classyfying top 20 wiki instances

In [27]:
instance_dict = {'Q5':'human',
'Q16521':'taxon',
'Q70208':'municip',
'Q8436':'human',
'Q8502':'geo',
'Q11879590':'human',
'Q13417114':'human',
'Q16970':'religion',
'Q55983715':'taxon',
'Q133056':'geo',
'Q5119':'municip',
'Q4022':'geo',
'Q158438':'settle',
'Q23442':'geo',
'Q28640':'prof',
'Q839954':'geo',
'Q101352':'human',
'Q11688446':'religion',
'Q15632617':'other',
'Q205985':'religion',
'Q2354973':'settle',
'Q46831':'geo',
'Q484170':'municip',
}

In [28]:
dfr['instance'] = dfr['instanceLabel'].map(instance_dict)

Label the not classified (< top 20) 'other'

In [29]:
dfr[['instance']] = dfr[['instance']].fillna('other')

In [30]:
dfr

,qid,name,instanceLabel,dateBirth,sexLabel,pic,sex,instance
0,Q63964,Sarnen Nord,Q5119,9999,kein,http://commons.wikimedia.org/wiki/Special:File...,,municip
0,Q64567,Buochs,Q70208,9999,kein,http://commons.wikimedia.org/wiki/Special:File...,,municip
0,Q63931,Stans Nord,Q70208,9999,kein,http://commons.wikimedia.org/wiki/Special:File...,,municip
0,Q124667,Tavelweg,Q5,1866-12-21T00:00:00Z,Q6581097,http://commons.wikimedia.org/wiki/Special:File...,male,human
0,Q1693700,Willadingweg,Q5,1641-01-01T00:00:00Z,Q6581097,http://commons.wikimedia.org/wiki/Special:File...,male,human
...,...,...,...,...,...,...,...,...
0,Q123886,Hintere Erlachstrasse,Q5,1299-01-01T00:00:00Z,Q6581097,http://commons.wikimedia.org/wiki/Special:File...,male,human
0,Q1495926,Hardungweg,Q5,1861-11-03T00:00:00Z,Q6581097,kein,male,human
0,Q78071721,Spühlweg,Q5,1916-09-03T00:00:00Z,Q6581097,kein,male,human
0,Q78071721,Spühlstrasse,Q5,1916-09-03T00:00:00Z,Q6581097,kein,male,human


## merge verification data to streetnames via name 

In [31]:
# Street name designations
streetnames = pd.read_csv('../Datapreparation/streetnames.csv', encoding='UTF-8-SIG', sep=';')

<font color='red'>WARNING</font> ambiguity of names is not considered. Could be considered by spatial restriction but was left out here. 

In [32]:
merge = pd.merge(dfr, streetnames, left_on=['name'], right_on=['STN_LABEL'])
merge

,qid,name,instanceLabel,dateBirth,sexLabel,pic,sex,instance,Unnamed: 0,STR_ESID,STN_LABEL,COM_FOSNR,COM_NAME,COM_CANTON,STR_OFFICIAL,STR_EASTING,STR_NORTHING,STN_LABEL_NO_BI,STN_LABEL_NO_TERMS,STR_TERMS,STR_PREPS,STN_LABEL_FINAL
0,Q124667,Tavelweg,Q5,1866-12-21T00:00:00Z,Q6581097,http://commons.wikimedia.org/wiki/Special:File...,male,human,3313,10022177,Tavelweg,2275,Murten,FR,True,2576509.0,1197547.0,Tavelweg,Tavel,weg,NaN,Tavel
1,Q124667,Tavelweg,Q5,1866-12-21T00:00:00Z,Q6581097,http://commons.wikimedia.org/wiki/Special:File...,male,human,78725,10089404,Tavelweg,387,Lengnau (BE),BE,True,2594575.0,1225189.0,Tavelweg,Tavel,weg,NaN,Tavel
2,Q124667,Tavelweg,Q5,1866-12-21T00:00:00Z,Q6581097,http://commons.wikimedia.org/wiki/Special:File...,male,human,80161,10117105,Tavelweg,356,Muri bei Bern,BE,True,2603562.0,1197746.0,Tavelweg,Tavel,weg,NaN,Tavel
3,Q124667,Tavelweg,Q5,1866-12-21T00:00:00Z,Q6581097,http://commons.wikimedia.org/wiki/Special:File...,male,human,83217,10012974,Tavelweg,979,Herzogenbuchsee,BE,True,2620038.0,1225679.0,Tavelweg,Tavel,weg,NaN,Tavel
4,Q124667,Tavelweg,Q5,1866-12-21T00:00:00Z,Q6581097,http://commons.wikimedia.org/wiki/Special:File...,male,human,83516,10198126,Tavelweg,880,Rüeggisberg,BE,True,2602847.0,1187735.0,Tavelweg,Tavel,weg,NaN,Tavel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4276,Q123886,Hintere Erlachstrasse,Q5,1299-01-01T00:00:00Z,Q6581097,http://commons.wikimedia.org/wiki/Special:File...,male,human,153401,10225984,Hintere Erlachstrasse,3203,St. Gallen,SG,True,2743189.0,1252886.0,Hintere Erlachstrasse,Erlach,strasse,hinter,Erlach
4277,Q1495926,Hardungweg,Q5,1861-11-03T00:00:00Z,Q6581097,kein,male,human,152103,10225832,Hardungweg,3203,St. Gallen,SG,True,2748092.0,1254605.0,Hardungweg,Hardung,weg,NaN,Hardung
4278,Q78071721,Spühlweg,Q5,1916-09-03T00:00:00Z,Q6581097,kein,male,human,165667,10246955,Spühlweg,3203,St. Gallen,SG,True,2749581.0,1256161.0,Spühlweg,Spühl,weg,NaN,Spühl
4279,Q78071721,Spühlstrasse,Q5,1916-09-03T00:00:00Z,Q6581097,kein,male,human,165964,10246954,Spühlstrasse,3203,St. Gallen,SG,True,2749624.0,1256261.0,Spühlstrasse,Spühl,strasse,NaN,Spühl


In [33]:
merge.to_csv(r'C:\CAS_Arbeit\cassda-zertifikatsarbeit\Evaluation\data\verif.csv', encoding='UTF-8-SIG', sep=';')